In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import reciprocal

from eis.EISDataIO import ECM_from_raw_strings
from utils_preprocessing import eis_dataframe_from_csv
from utils_preprocessing import interpolate_to_freq_range, sort_circuits

from utils import plot_freq_range, meas_points_print
from utils import umap_plots, plot_all_nyquist

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [ ]:
def print_information(df, df_test):
    print(f"Train data:{len(df)} spectra")
    print(f"Test data:{len(df_test)} spectra")
    print(f"Total data:{len(df) + len(df_test)} spectra")

    df_all = pd.concat([df, df_test])
    for circuit in df_all["Circuit"].unique():
        print(f"{circuit}: {len(df_all[df_all['Circuit'] == circuit])}")

In [ ]:
# Load the csv into a data frame.
filtered = 1
save_figs = 1
subplots_spectra = 0
df_f = eis_dataframe_from_csv("data/train_data_filtered.csv")
df_test_f = eis_dataframe_from_csv("data/test_data_filtered.csv")
# Print the number of spectra per circuit in the data frame.
print_information(df_f, df_test_f)

df_uf = eis_dataframe_from_csv("data/train_data.csv")
df_test_uf = eis_dataframe_from_csv("data/test_data.csv")
# Print the number of spectra per circuit in the data frame.
print_information(df_uf, df_test_uf)

if filtered: 
    df = df_f.copy()
    df_test = df_test_f.copy()
else:
    df = df_uf.copy()
    df_test = df_test_uf.copy()

In [ ]:
# The parameter values are drawn from independent reciprocal distributions.
# They are a bit tricky to visualize with histograms, thus here's an example:
def plot_stats(array, ax, cirucit, param_name, bins=20):
    ax.set_title(f"{cirucit} {param_name}")
    ax.set_xlabel("Parameter value")
    # Set x axis to log scale.
    ax.set_xscale("log")
    ax.set_ylabel("Count")
    ax.hist(array, bins=bins)
    return ax 


a, b = 10e-8, 10e-5
mean, var, skew, kurt = reciprocal.stats(a, b, moments='mvsk')
x = np.linspace(reciprocal.ppf(0.01, a, b), reciprocal.ppf(0.99, a, b), 100)

fig, ax = plt.subplots(1, 1)
ax.plot(x, reciprocal.pdf(x, a, b), 'r-', lw=1, alpha=0.6, label='reciprocal pdf')
ax.set_title("Reciprocal distribution, PDF")

fig, ax = plt.subplots(1, 1)
r = reciprocal.rvs(a, b, size=500)
plot_stats(r, ax, "", "Reciprocal Test")

fig, ax = plt.subplots(1, 1)
r = reciprocal.rvs(1.59, 10.1, size=300)
# round r to 3 decimals
r = np.round(r, 3)
plot_stats(r, ax, "", "Reciprocal Test")

# Now draw from a uniform distribution. 
fig, ax = plt.subplots(1, 1)
u = np.random.uniform(0.5, 1.01, size=1000)
# round r to 3 decimals
u = np.round(u, 3)
plot_stats(u, ax, "", "Uniform Test")

# Now draw from a uniform distribution. 
fig, ax = plt.subplots(1, 1)
rt = reciprocal.rvs(0.5, 1.01, size=1000)
# round r to 3 decimals
rt = np.round(rt, 3)
plot_stats(rt, ax, "", "Reciprocal Test")

In [ ]:
# Get all the parameter distributions of the circuits.

# Merge the data frames.
df_all = df.append(df_test, ignore_index=True)
# Sort 
df_all = sort_circuits(df_all)

# turn string into dict of str float pairs
parameter_dict = {}
max_nb_params = 0
for circuit in df_all["Circuit"].unique():
    param_names = ECM_from_raw_strings(circuit, df_all.loc[df_all["Circuit"]==circuit].iloc[0].Parameters).param_names
    # make empty pandas df with param_names as columns
    parameter_dict[circuit] = pd.DataFrame(columns=param_names)
    if len(param_names) > max_nb_params:
        max_nb_params = len(param_names)
    # add rows with param_values
    for i, sample in enumerate(df_all.loc[df_all["Circuit"]==circuit].itertuples()):
        parameter_dict[circuit].loc[i] = ECM_from_raw_strings(sample.Circuit, sample.Parameters).param_values

# Check the covariance between the columns 
for circuit in df_all["Circuit"].unique():  
    print(parameter_dict[circuit].corr())
    

fig, ax = plt.subplots(9, max_nb_params, figsize=(45, 45))
for i, circuit in enumerate(df_all["Circuit"].unique()):
    for j, param_name in enumerate(parameter_dict[circuit].columns):
        ax[i, j] = plot_stats(parameter_dict[circuit][param_name].values.flatten(), ax[i, j], circuit, param_name)
fig.tight_layout()

# Conclusions:
# 1. Parameters are independent
# 2. Parameters are drawn from a reciprocal distribution, but are rounded to 3 significant digits.
# 3. The time constants are drwan form a unifrom distribution.

In [ ]:
np.min(parameter_dict["RC-RC-RCPE-RCPE"]["C1"])

In [ ]:
np.max(parameter_dict["RC-RC-RCPE-RCPE"]["C1"])

In [ ]:
# Get the minimum distance between the time constants of spectra with multiple time constants.

# Circuit with multiple time constants: 
circuits_multiple_t = ["L-R-RCPE-RCPE", "L-R-RCPE-RCPE-RCPE", "RC-RC-RCPE-RCPE", "RCPE-RCPE", "RCPE-RCPE-RCPE", "RCPE-RCPE-RCPE-RCPE"]
column_names = [["CPE1_t", "CPE2_t"], ["CPE1_t", "CPE2_t", "CPE3_t"], ["tau1", "tau2", "CPE1_t", "CPE2_t"], ["CPE1_t", "CPE2_t"], ["CPE1_t", "CPE2_t", "CPE3_t"], ["CPE1_t", "CPE2_t", "CPE3_t", "CPE4_t"]]
# count the letter C in the circuit string
count_C = lambda x: x.count("C")

parameter_dict_time_constants = {}
for i, circuit in enumerate(circuits_multiple_t):
    t_ind = 0 
    parameter_dict_time_constants[circuit] = pd.DataFrame(columns=column_names[i])
    print(f"{circuit}")
    for j, param_name in enumerate(parameter_dict[circuit].columns):
        if circuit == "RC-RC-RCPE-RCPE":
            if param_name=="R1":
                tau = parameter_dict[circuit][param_name].values.flatten() * parameter_dict[circuit]["C1"].values.flatten()
                parameter_dict_time_constants[circuit][column_names[i][t_ind]] = tau
                t_ind += 1
            if param_name=="R2":
                tau = parameter_dict[circuit][param_name].values.flatten() * parameter_dict[circuit]["C2"].values.flatten()
                parameter_dict_time_constants[circuit][column_names[i][t_ind]] = tau
                t_ind += 1
        if param_name[-1] == "t":
            print(f"{param_name} {parameter_dict[circuit][param_name].values.flatten()}")
            parameter_dict_time_constants[circuit][column_names[i][t_ind]] = parameter_dict[circuit][param_name].values.flatten()
            t_ind += 1
        # ax[i, j] = plot_stats(parameter_dict[circuit][param_name].values.flatten(), ax[i, j], circuit, param_name)



In [ ]:
parameter_dict_time_constants[circuit]

In [ ]:
from itertools import combinations 

for key in parameter_dict_time_constants.keys():
    tds = list(combinations(parameter_dict_time_constants[key].columns,2))
    td_diff= pd.concat([np.abs(parameter_dict_time_constants[key][c[1]].sub(parameter_dict_time_constants[key][c[0]])) for c in tds], axis=1, keys=tds)
    # appned td_diff columns to parameter_dict_time_constants[key]
    td_diff.columns = td_diff.columns.map('-'.join)
    # add "diff" to the column names
    td_diff.columns = [f"diff{c}" for c in td_diff.columns]
    parameter_dict_time_constants[key] = pd.concat([parameter_dict_time_constants[key], td_diff], axis=1)

# draw 1000 values uniform at random 
t_example = np.random.uniform(0.5, 1.0, size=1000)
t2_example = np.random.uniform(0.5, 1.0, size=1000)
td_example = np.abs(t_example - t2_example)

fig, ax = plt.subplots(1, 1)
ax.hist(td_example, bins=50)

In [ ]:
min_diff_taus = np.min(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["difftau1-tau2"])
print(f"Min difference {min_diff_taus}")
diff_taus = np.sort(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["difftau1-tau2"])
print(f"Sorted differences {diff_taus[:10]}")

median_diff_taus = np.median(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["difftau1-tau2"])
print(f"Median difference {median_diff_taus}")
mean_diff_taus = np.mean(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["difftau1-tau2"])
print(f"Mean difference {mean_diff_taus}")

In [ ]:
min_diff_ts = np.min(parameter_dict_time_constants["L-R-RCPE-RCPE"]["diffCPE1_t-CPE2_t"])
print(f"Min difference {min_diff_ts}")
diff_ts = parameter_dict_time_constants["L-R-RCPE-RCPE"]["diffCPE1_t-CPE2_t"]
diff_ts = np.sort(diff_ts)
print(f"Sorted differences {diff_ts[:10]}")
median_diff_ts = np.median(parameter_dict_time_constants["L-R-RCPE-RCPE"]["diffCPE1_t-CPE2_t"])
print(f"Median difference {median_diff_ts}")
mean_diff_ts = np.mean(parameter_dict_time_constants["L-R-RCPE-RCPE"]["diffCPE1_t-CPE2_t"])
print(f"Mean difference {mean_diff_ts}")

In [ ]:
fig, ax = plt.subplots(6, 6, figsize=(20, 20))
for i, key in enumerate(parameter_dict_time_constants.keys()):
    # Gett all the columns with "diff" in the name
    diff_cols = [col for col in parameter_dict_time_constants[key].columns if "diff" in col]
    print(diff_cols)
    ax[i, 0].set_ylabel(f"{key} \n Count")
    for j, col in enumerate(diff_cols):
        ax[i, j].set_title(f"|{col[4:]}|")
        ax[i, j].hist(parameter_dict_time_constants[key][col].values.flatten(), bins=50)
        if col == "difftau1-tau2":
            # set the axis to log scale
            ax[i, j].set_xscale("log")
        #ax[i, j] = plot_stats(parameter_dict_time_constants[key][col].values.flatten(), ax[i, j], "", col, bins=50)
    if j < 6:
        for k in range(j+1, 6):
            ax[i, k].set_visible(False)
fig.tight_layout()
# ax[i, j].set_title("Distance between time constants")

In [ ]:
tau1 = parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"]
tau2 = parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"]

tr = [tau1/tau2 if tau1/tau2>1 else tau2/tau1 for tau1, tau2 in zip(tau1, tau2)]

print(f"Min ratio {np.min(tr)}")
print(f"Max ratio {np.max(tr)}")

In [ ]:
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"]

In [ ]:
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["td"] = np.abs(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"] - parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"])
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr1"] = parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"] / parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"]
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr2"] = parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"] / parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"]



print(np.min(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr2"]))
print(np.max(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr2"]))

print(np.min(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr1"]))
print(np.max(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr1"]))

In [ ]:
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["td"] = np.abs(parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"] - parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"])
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr1"] = parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"] / parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"]
parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tr2"] = parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau2"] / parameter_dict_time_constants["RC-RC-RCPE-RCPE"]["tau1"]

# Get the minimum distance between the time constants of spectra with multiple time constants.

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# mak a histogram plot of the distance between the time constants
ax.set_title("Distance between time constants")
ax.set_xlabel("Distance")
ax.set_ylabel("Count")
ax.hist(parameter_dict_time_constants["RCPE-RCPE"]["td"].values.flatten(), bins=50)

In [ ]:
parameter_dict_time_constants["L-R-RCPE-RCPE"]["td"] = np.abs(parameter_dict_time_constants["L-R-RCPE-RCPE"]["t0"] - parameter_dict_time_constants["L-R-RCPE-RCPE"]["t1"])
# Get the minimum distance between the time constants of spectra with multiple time constants.

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# mak a histogram plot of the distance between the time constants
ax.set_title("Distance between time constants")
ax.set_xlabel("Distance")
ax.set_ylabel("Count")
ax.hist(parameter_dict_time_constants["L-R-RCPE-RCPE"]["td"].values.flatten(), bins=50)


In [ ]:
print(np.min(parameter_dict_time_constants["L-R-RCPE-RCPE"]["td"]))
print(np.max(parameter_dict_time_constants["L-R-RCPE-RCPE"]["td"]))

In [ ]:
# legend
# Font size
plot_freq_range(df, save=save_figs, verbose=False)

In [ ]:
# Include legend 
meas_points_print(df)

In [ ]:
# Conclusion: There is no correspondance between the measurement points and the circuit 
# and there is no correspondance between the frequency ranges and the circuit.

# Nevertheless, to turn the data into a matrix for machine learning, we need to interpolate to
# a common frequency range (i.e., constant frequency values for all samples).
interpolated_basis = np.geomspace(10, 1e5, num=30)

df_sorted = sort_circuits(interpolate_to_freq_range(df, interpolated_basis))
df_test_sorted = sort_circuits(interpolate_to_freq_range(df_test, interpolated_basis))

In [ ]:
# Umap random state fixed (plots differ slightly from publication, because the seed was previously not fixed).
# Even though the seed is fixed, subple changes can occur (probabaly due to the stochastic nature of UMAP.)
umap_plots(df_sorted, save=save_figs, random_state=42)

In [ ]:
# Visualize data as images (used for the CNN)
if subplots_spectra:
    rows = 67 
    cols = 111
    fig, axs = plt.subplots(rows, cols, sharex=False, sharey=False, figsize=(4*cols, 4*rows), frameon=False)
    for i in range(rows):
        if i == 0:
            axs[i, 0].set_ylabel("zreal")
        for j in range(cols):
            axs[i, j].set_axis_off()
            axs[i, j].plot(df['zreal'][i*cols+j], -df['zimag'][i*cols+j], linewidth=3, color='black')

    axs[i, j].set_xlabel('Frequency (Hz)')
    if save_figs:
        fig.savefig('figures/eis_art.pdf')
    plt.show()

In [ ]:
# Showcase outliers by plotting all spectra in a single plot
ax = plot_all_nyquist(df, x_name='zreal', y_name='zimag', title='Spectra Labeled Data')

In [ ]:
# Normalize all spectra such that zreal and -zimag are in the range [0, 1]

df['zreal_norm'] = df['zreal'].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df['zimag_norm'] = df['zimag'].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df['zreal_norm2'] = df['zreal'].apply(lambda x: (x) / (x.max()))
df['zimag_norm2'] = df['zimag'].apply(lambda x: (x) / (- x.min()))

ax = plot_all_nyquist(
    df,
    x_name='zreal_norm',
    y_name='zimag_norm',
    title='Spectra Labeled Data (Normalized)',
    linewidth=0.022,
    save_name='eis_art2_norm',
)

In [ ]:
lw = 0.1
alpha = 0.2
label_fontsize = 16
# 5, 2 , 10, 23
# Make a 5 row 2 column subplot
rows = 3
cols = 3
fig, axs = plt.subplots(rows, cols, sharex=True, sharey=True, subplot_kw=dict(box_aspect=1), frameon=False)
# Set the figure size
fig.set_size_inches(15, 15)
# unique circuits
circuits = np.sort(df['Circuit'].unique())
# Iterate over the axes and the circuits
for i, ax in enumerate(axs.flat):
    ax.set_aspect('equal')
    if i <= len(circuits):
        # get df for circuit
        df_circuit = df[df['Circuit'] == circuits[i]].copy()
        ax = plot_all_nyquist(
            df_circuit,
            ax,
            'zreal_norm',
            'zimag_norm',
            title=circuits[i],
            linewidth=lw,
            alpha=alpha,
            label_fontsize=label_fontsize
        )
        # if i even remove y labels
        if i % cols != 0:
            ax.set_ylabel('')
            # Remove y ticks
            ax.set_yticks([])
        # if i not in last row remove x labels
        if i not in [i for i in range(len(circuits)) if i+1 > len(circuits) - cols]:
            ax.set_xlabel('')
            # Remove x ticks
            ax.set_xticks([])
    else:
        ax.set_axis_off()

fig.set_tight_layout(True)
fig.savefig('figures/spiderweb_eis_circuits.pdf')
fig.savefig('figures/spiderweb_eis_circuits.eps')
fig.savefig('figures/spiderweb_eis_circuits.png', dpi=300)